In [1]:
%pip install pandas sentence-transformers pinecone-client transformers tf-keras bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 23.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/indian-food-dataset/Indian_Food_Dataset.csv")
display(df.head())

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon m...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seeds...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...


In [3]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "recipe-index"
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=384,spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        ) )
index = pc.Index(index_name)


model = SentenceTransformer('all-MiniLM-L6-v2')
for i, row in df.iterrows():
    combined_text = f"Ingredients: {row['Cleaned-Ingredients']}\nInstructions: {row['TranslatedInstructions']}"
    embedding = model.encode(combined_text)
    metadata = {
        "recipe_name": row['TranslatedRecipeName'],
        "ingredients": row['Cleaned-Ingredients'],
        "measurements": row['TranslatedIngredients'],
        "prep_time": row['TotalTimeInMins'],
        "instructions": row['TranslatedInstructions'],
        "cuisine": row['Cuisine'],
        "url": row['URL'],
        "final_look": row['image-url']
    }
    index.upsert([(str(i), embedding, metadata)])

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'dadf2ed550af376500f711334093245b', 'Date': 'Mon, 06 Jan 2025 04:11:12 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
def query_recipes(user_query, model=SentenceTransformer('all-MiniLM-L6-v2')):
    # Reconnect to Pinecone
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index("recipe-index")
    
    query_embedding = model.encode(user_query)
    # Use keyword arguments for the query
    results = index.query(
        vector=query_embedding.tolist(),  # specify 'vector=' explicitly
        top_k=3,
        include_metadata=True
    )
    return results

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from huggingface_hub import login

# Login with your Hugging Face token
login(HF_TOKEN)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
%pip install -U bitsandbytes

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model with auto device map and quantization
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",  # Automatically set devices
    low_cpu_mem_usage=True
)

# Generate a response based on retrieved recipes
def generate_response(user_query, retrieved_recipes):
    context = "Relevant Recipes:\n"
    for recipe in retrieved_recipes['matches']:
        metadata = recipe['metadata']
        context += (
            f"Recipe Name: {metadata['recipe_name']}\n"
            f"Time: {metadata['prep_time']}\n"
            f"Ingredients: {metadata['ingredients']}\n"
            f"Measurements: {metadata['measurements']}\n"
            f"Instructions: {metadata['instructions']}\n\n"
        )
    prompt = context + f"User Query: {user_query}"

    # Move inputs to the same device as the model
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    max_input_length = tokenizer.model_max_length - 200
    if len(inputs['input_ids'][0]) > max_input_length:
        truncated_prompt = tokenizer.decode(inputs['input_ids'][0][:max_input_length], skip_special_tokens=True)
        inputs = tokenizer(truncated_prompt, return_tensors="pt").to(device)

    # Generate output
    outputs = model.generate(**inputs, max_new_tokens=300)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

user_query = "What can I cook with chicken and tomato?"
retrieved_recipes = query_recipes(user_query)  # Ensure query_recipes is defined
if retrieved_recipes and "matches" in retrieved_recipes:
    response = generate_response(user_query, retrieved_recipes)
    print("Final Response:")
    print(response)
else:
    print("No relevant recipes found.")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Final Response:
Relevant Recipes:
Recipe Name: Chicken Parmigiana Recipe
Time: 70.0
Ingredients: tomato,salt,rosemary,eggs,tabasco original sauce,flour (maida),mozzarella cheese,cloves garlic,red chilli powder,red chilli flakes,sugar,onion,pepper,parmesan cheese,chicken breasts,wheat bread crumbs,sunflower oil
Measurements: 1 teaspoon Sugar,Salt - to taste,1 teaspoon Red Chilli powder,1/2 cup Whole Wheat Bread crumbs,1 Whole Egg,1 Onion - chopped,1 teaspoon Red Chilli flakes,1 teaspoon Rosemary,1 teaspoon Salt and Pepper,4 tablespoon All Purpose Flour (Maida),1/8 cup Parmesan cheese - grated,4 cloves Garlic - chopped,Sunflower Oil,1 teaspoon Tabasco Original - Hot Sauce,2 cups Homemade tomato puree,1 Chicken breasts - cut diagonally thin,1/3 cup Mozzarella cheese - grated
Instructions: To begin making the Chicken Parmigiana Recipe, thoroughly wash and clean the chicken and cut them into 2 inch cubes.
To make the sauceHeat a sauce pan with oil on medium flame, add garlic and fry until i

In [8]:
user_query = "What can I cook with chicken and tomato?"
retrieved_recipes = query_recipes(user_query)
if retrieved_recipes and "matches" in retrieved_recipes:
    response = generate_response(user_query, retrieved_recipes)
    print("Final Response:")
    print(response)
else:
    print("No relevant recipes found.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Final Response:
Relevant Recipes:
Recipe Name: Chicken Parmigiana Recipe
Time: 70.0
Ingredients: tomato,salt,rosemary,eggs,tabasco original sauce,flour (maida),mozzarella cheese,cloves garlic,red chilli powder,red chilli flakes,sugar,onion,pepper,parmesan cheese,chicken breasts,wheat bread crumbs,sunflower oil
Measurements: 1 teaspoon Sugar,Salt - to taste,1 teaspoon Red Chilli powder,1/2 cup Whole Wheat Bread crumbs,1 Whole Egg,1 Onion - chopped,1 teaspoon Red Chilli flakes,1 teaspoon Rosemary,1 teaspoon Salt and Pepper,4 tablespoon All Purpose Flour (Maida),1/8 cup Parmesan cheese - grated,4 cloves Garlic - chopped,Sunflower Oil,1 teaspoon Tabasco Original - Hot Sauce,2 cups Homemade tomato puree,1 Chicken breasts - cut diagonally thin,1/3 cup Mozzarella cheese - grated
Instructions: To begin making the Chicken Parmigiana Recipe, thoroughly wash and clean the chicken and cut them into 2 inch cubes.
To make the sauceHeat a sauce pan with oil on medium flame, add garlic and fry until i

In [ ]:
user_query = "What can I cook left over rice?"
retrieved_recipes = query_recipes(user_query)
if retrieved_recipes and "matches" in retrieved_recipes:
    response = generate_response(user_query, retrieved_recipes)
    print("Final Response:")
    print(response)
else:
    print("No relevant recipes found.")

In [ ]:
user_query = "What can I cook avacado and eggs?"
retrieved_recipes = query_recipes(user_query)
if retrieved_recipes and "matches" in retrieved_recipes:
    response = generate_response(user_query, retrieved_recipes)
    print("Final Response:")
    print(response)
else:
    print("No relevant recipes found.")